In [ ]:
conda install pip

In [ ]:
pip install --upgrade tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np
import argparse
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
pip install opencv-python


In [ ]:
a = int(input("Which mode?"))
if (a==0):
    mode = "train"
else:
    mode = "display"

In [ ]:
def rgb2gray(rgb):
                    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [ ]:
num_train = 28709
num_val = 7178
batch_size = 64
num_epoch = 25

train_dir = 'train'
val_dir = 'test'

LABELS = [ "angry", "anxiety", "happy", "neutral", "sad"]
X_TRAIN = []
Y_TRAIN = []

for label in LABELS:
        path = os.path.join(train_dir, label)
        class_num = LABELS.index(label)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_array = rgb2gray(img_array)
                X_TRAIN.append(img_array)
                Y_TRAIN.append(class_num)
            except Exception as e:
                pass

X_TRAIN=np.asarray(X_TRAIN, np.uint8)
print(X_TRAIN)
number_of_classes = len(set(Y_TRAIN))  
Y_TRAIN = np.array(Y_TRAIN)
Y_TRAIN = Y_TRAIN.astype('float32')
print(Y_TRAIN)
Y_TRAIN = to_categorical(Y_TRAIN, number_of_classes)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

if mode == "train":
    model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])
    model_info = model.fit(X_TRAIN,Y_TRAIN,epochs=num_epoch,validation_data=validation_generator,validation_steps=num_val // batch_size)
    model.save_weights('model.h5')

elif mode == "display":
    model.load_weights('model.h5')

    cv2.ocl.setUseOpenCL(False)
    emotion_dict = {0:"angry",1:"anxiety",2:"happy",3:"neutral",4:"sad"}

    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_gray = gray[y:y + h, x:x + w]
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
            prediction = model.predict(cropped_img)
            print(prediction)
            maxindex = int(np.argmax(prediction))
            text = "{}: {:.2f}%".format(emotion_dict[maxindex], np.max(prediction) * 100)

            
            cv2.putText(frame, text, (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()